---
title: "2023_Kronensicherung_Plesse_004_Combination_and_Cleaning"
author: "Kyell Jensen"
date: "2024-08-06"
format: pdf
editor: visual
---

# 2023_Kronensicherung_Plesse_Combination_and_Cleaning

## Kombinieren und Bereinigen der Daten von LineScale3, TreeQinetic und Versuchsaufzeichung

Nutze eine geeignete Python 3.11 Umgebung (z. B. virtuelle Environment).

## Arbeitsumgebung vorbereiten

### IMPORT: Importieren von Standardbibliotheken

Die folgenden Bibliotheken werden importiert, um grundlegende Funktionen für Strukturierung, Datenverarbeitung, Plotting und statistische Auswertung bereit zu stellen.

In [1]:
# Struktur
from pathlib import Path
from typing import Dict, List

# Datenverarbeitung
import json
from IPython.display import Markdown, display
import numpy as np
import pandas as pd
from pandas.api.types import CategoricalDtype

### IMPORT: Importiere eigene Pakete

Lade allgemeine Export-Funktionen, um die Daten als Latex-Tabellen zu exportieren

In [2]:
from kj_core.utils.latex_export import (
    save_latex_table,
    build_data_dict_df
)

## IMPORT: Daten Import

Lege Pfade für Daten-Importe, Daten-Exporte etc. fest (ggf. anpassen an eigene Verzeichnisstruktur), ausgelagert in gemeinsame Config für verschiedene Notebooks

In [3]:
# Importiere alle Einstellungen aus der project_config.py
from project_config import (
    data_path,
    data_export_directory,
    latex_export_directory
)

In [4]:
# Liste aller Dateinamen
filenames = [
    "tree.feather",
    "sensor.feather",
    "sensor_data_dict.json",
    "series.feather",
    "series_data_dict.json",
    "ls3_data_dict.json",
    "ls3.feather",
    "ptq_data_dict.json",
    "ptq.feather",
]

In [5]:
def load_file(path: Path):
    try:
        if path.suffix == ".feather":
            df_now = pd.read_feather(path)
            print(f"[Info] Feather geladen: {path.name}")
            return "df", path.stem, df_now
        elif path.suffix == ".json":
            with open(path, "r", encoding="utf-8") as f:
                data_dict_now = json.load(f)
                print(f"[Info] JSON geladen: {path.name}")
                return "dict", path.stem, data_dict_now
        else:
            print(f"[Warnung] Unbekannter Dateityp übersprungen: {path.name}")
            return None
    except Exception as e:
        print(f"[Fehler] Laden fehlgeschlagen für '{path.name}': {e}")
        return None


In [6]:
# Container
dfs = {}
data_dicts = {}

for file in filenames:
    result = load_file(data_export_directory / file)
    if result:
        kind, name_stem, content = result
        if kind == "df":
            dfs[f"{name_stem}_df"] = content
        elif kind == "dict":
            data_dicts[f"{name_stem}"] = content

[Info] Feather geladen: tree.feather
[Info] Feather geladen: sensor.feather
[Info] JSON geladen: sensor_data_dict.json
[Info] Feather geladen: series.feather
[Info] JSON geladen: series_data_dict.json
[Info] JSON geladen: ls3_data_dict.json
[Info] Feather geladen: ls3.feather
[Info] JSON geladen: ptq_data_dict.json
[Info] Feather geladen: ptq.feather


In [7]:
dfs.keys()

dict_keys(['tree_df', 'sensor_df', 'series_df', 'ls3_df', 'ptq_df'])

In [8]:
data_dicts.keys()

dict_keys(['sensor_data_dict', 'series_data_dict', 'ls3_data_dict', 'ptq_data_dict'])

In [9]:
# IDE-Hints: Feather-Dateien (DataFrames)
tree_df: pd.DataFrame = dfs.get("tree_df")
sensor_df: pd.DataFrame = dfs.get("sensor_df")
series_df: pd.DataFrame = dfs.get("series_df")
ls3_df: pd.DataFrame = dfs.get("ls3_df")
ptq_df: pd.DataFrame = dfs.get("ptq_df")

# IDE-Hints: Data Dictionary Dateien (JSON → dict)
sensor_data_dict: dict = data_dicts.get("sensor_data_dict")
series_data_dict: dict = data_dicts.get("series_data_dict")
ls3_data_dict: dict = data_dicts.get("ls3_data_dict")
ptq_data_dict: dict = data_dicts.get("ptq_data_dict")


In [10]:
ptq_df

,id,file_name,sensor_name,sample_rate,max_strain,max_compression,max_strain_osc,max_compression_osc,m_amplitude,m_amplitude_2,...,rmse,nrmse,cv,mae,nmae,integral_intercept,integral_positiv,integral_negativ,integral_abs,integral_ratio
0,1,PTQ_Meas_100346.txt,Elasto(90),4.003157,368.0,-168.1,363.5,-168.1,265.80,162.10,...,24.382157,0.045868,-60.280300,10.359435,0.019488,-1.870831,391.325732,365.341989,756.667721,1.071122
1,1,PTQ_Meas_100346.txt,Elasto(92),3.941160,240.9,-86.6,239.9,-86.6,163.25,76.00,...,13.313315,0.040776,-1.212033,6.809497,0.020856,-10.571599,174.631434,182.059739,356.691173,0.959199
2,1,PTQ_Meas_100346.txt,Elasto(95),4.003157,215.0,-119.1,190.2,-119.1,154.65,82.25,...,14.852713,0.048022,-0.520646,8.316242,0.026888,-30.255247,222.431092,191.815217,414.246308,1.159611
3,1,PTQ_Meas_100346.txt,Elasto(98),3.941160,192.8,-132.8,141.6,-132.8,137.20,92.50,...,8.212944,0.029932,-0.399388,5.589269,0.020370,-21.122985,232.922135,222.857242,455.779377,1.045163
4,2,PTQ_Meas_101814.txt,Elasto(90),3.447691,429.6,-202.2,417.5,-202.2,309.85,162.95,...,24.083544,0.038866,-3.478901,11.160700,0.018011,-3.467562,258.909157,319.996801,578.905957,0.809099
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111,28,PTQ_Meas_155805.txt,Elasto(98),2.391119,149.1,-108.4,120.3,-108.4,114.35,95.15,...,9.722030,0.042510,-1.489630,6.965469,0.030457,-8.622783,168.196780,131.385606,299.582386,1.280177
112,29,PTQ_Meas_160907.txt,Elasto(90),2.967190,387.9,-94.4,383.8,-94.4,239.10,107.30,...,36.498548,0.076325,12.131841,16.258373,0.033999,-11.971655,418.627008,156.174828,574.801836,2.680502
113,29,PTQ_Meas_160907.txt,Elasto(92),2.967190,263.4,-28.3,259.7,-22.0,140.85,39.30,...,22.091387,0.078422,5.346908,10.226893,0.036304,-6.016494,231.918646,54.123696,286.042342,4.284974
114,29,PTQ_Meas_160907.txt,Elasto(95),2.967190,237.6,-23.9,228.1,-23.9,126.00,46.20,...,20.255176,0.080378,3.804639,9.617778,0.038166,-2.985686,208.082876,62.500514,270.583390,3.329299


In [11]:
ls3_df

,measurement_name,sensor_id,datetime,measurement_id,unit,mode,rel_zero,speed,trig,stop,...,duration,length,max_index,max_force,min_index,min_force,mean,median,integral,release
0,14_41_10,14:99:1E,2022-03-23 14:41:10,1,kN,ABS,n/a,1280,0.8,0.0,...,17.999218,23040,4737,1.48,36,0.00,0.111598,0.060,1.114957,NaN
1,14_47_59,14:99:1E,2022-03-23 14:47:59,2,kN,ABS,n/a,1280,0.8,0.0,...,17.999218,23040,4721,1.44,135,0.00,0.117817,0.060,1.188226,NaN
2,14_55_03,14:99:1E,2022-03-23 14:55:03,3,kN,ABS,n/a,1280,0.8,0.0,...,17.999218,23040,3867,1.33,81,0.00,0.104375,0.060,0.912268,NaN
3,15_04_05,14:99:1E,2022-03-23 15:04:05,4,kN,ABS,n/a,1280,0.8,0.0,...,17.999218,23040,3861,1.01,49,-0.01,0.072250,0.040,0.725232,NaN
4,15_08_55,14:99:1E,2022-03-23 15:08:55,5,kN,ABS,n/a,1280,0.8,0.0,...,17.999218,23040,4730,1.16,73,0.00,0.108828,0.060,0.980189,NaN
5,15_12_57,14:99:1E,2022-03-23 15:12:57,6,kN,ABS,n/a,1280,0.8,0.0,...,17.999218,23040,4695,1.16,1146,-0.01,0.080535,0.040,0.874301,NaN
6,15_16_58,14:99:1E,2022-03-23 15:16:58,7,kN,ABS,n/a,1280,0.8,0.0,...,17.999218,23040,4670,1.01,4,0.00,0.082550,0.040,0.814691,NaN
7,15_21_49,14:99:1E,2022-03-23 15:21:49,8,kN,ABS,n/a,1280,0.8,0.0,...,17.999218,23040,4684,1.15,16,0.00,0.090246,0.050,0.871205,NaN
8,15_25_17,14:99:1E,2022-03-23 15:25:17,9,kN,ABS,n/a,1280,0.8,0.0,...,17.999218,23040,4665,1.03,6,0.00,0.083470,0.050,0.720548,NaN
9,16_11_40,14:99:1E,2022-03-23 16:11:40,10,kN,ABS,n/a,1280,1.0,0.0,...,17.999218,23040,3884,2.59,94,0.00,0.404725,0.440,1.321706,NaN


In [12]:
sensor_df

,position_id,type,sensor_id,location,height,circumference,direction,note,diameter
0,1,LS3,14:BF:E6,rope,18.3,<NA>,<NA>,<NA>,<NA>
1,2,LS3,14:99:1E,cable,17.4,<NA>,<NA>,<NA>,<NA>
2,3,TMS1,015,StA,18.0,0.45,west,<NA>,0.143239
3,4,TMS1,014,StB,18.0,0.4,west,<NA>,0.127324
4,5,TMS1,013,StA,15.0,0.67,west,<NA>,0.213268
5,6,TMS1,012,StB,15.0,0.56,west,<NA>,0.178254
6,7,TMS1,011,StA,12.0,0.77,west,<NA>,0.245099
7,8,TMS1,010,StB,12.0,0.76,west,<NA>,0.241916
8,9,TMS1,007,StAB,9.0,1.14,west,<NA>,0.362873
9,10,Elasto,98,StA,16.85,0.64,elongation,moved_measurement_4,0.203718


## MERGE: Zusammenführen der Daten von LS3, PTQ und Versuchsprotokoll

Beginne mit PTQ-Daten, füge dann schrittweise die Daten des Versuchsablaufes, der Sensorpositionierung und Kraftmessung hinzu

In [13]:
df = pd.merge(ptq_df, series_df, on='id', how='left')
# df

Ergänze die Daten der Sensorpositionierung

In [14]:
# Merging df and elasto_df
# Geräte vom Typ Elastometer auswählen aus dem sensor_df
elasto_df = sensor_df[sensor_df["type"] == "Elasto"].copy()

# Passe die sensor_namen einheitlich für beide DataFrames an
elasto_df["sensor_name"] = elasto_df["type"].astype(str) + "(" + elasto_df["sensor_id"].astype(str) + ")"

# Perform the left join on sensor_name
df = df.merge(elasto_df, on="sensor_name", how="left")

# Gewünschte Reihenfolge
elasto_names = ["Elasto(90)", "Elasto(92)", "Elasto(95)", "Elasto(98)"]
# Definieren als geordneter CategoricalDtype
elasto_cat_type = CategoricalDtype(categories=elasto_names, ordered=True)
# Setzen des Typs auf die Spalte
df["sensor_name"] = df["sensor_name"].astype(elasto_cat_type)
# df

Ergänze die LS3-Daten

In [15]:
# Function to prepare ls3_metadata
def extract_ls3_subset(df, sensor_id, new_prefix):
    return (df.query(f"sensor_id == '{sensor_id}'")
            .drop('sensor_id', axis=1)
            .add_prefix(new_prefix)
            .rename(columns={f'{new_prefix}measurement_id': sensor_id}))

# Merging ls3_metadata with the main DataFrame
df = df.merge(extract_ls3_subset(ls3_df, '14:BF:E6', 'rope_'), on='14:BF:E6', how='left')     
df = df.merge(extract_ls3_subset(ls3_df, '14:99:1E', 'cable_'), on='14:99:1E', how='left')     

In [16]:
df

,id,file_name,sensor_name,sample_rate,max_strain,max_compression,max_strain_osc,max_compression_osc,m_amplitude,m_amplitude_2,...,cable_duration,cable_length,cable_max_index,cable_max_force,cable_min_index,cable_min_force,cable_mean,cable_median,cable_integral,cable_release
0,1,PTQ_Meas_100346.txt,Elasto(90),4.003157,368.0,-168.1,363.5,-168.1,265.80,162.10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,PTQ_Meas_100346.txt,Elasto(92),3.941160,240.9,-86.6,239.9,-86.6,163.25,76.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,PTQ_Meas_100346.txt,Elasto(95),4.003157,215.0,-119.1,190.2,-119.1,154.65,82.25,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,PTQ_Meas_100346.txt,Elasto(98),3.941160,192.8,-132.8,141.6,-132.8,137.20,92.50,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2,PTQ_Meas_101814.txt,Elasto(90),3.447691,429.6,-202.2,417.5,-202.2,309.85,162.95,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111,28,PTQ_Meas_155805.txt,Elasto(98),2.391119,149.1,-108.4,120.3,-108.4,114.35,95.15,...,17.999218,23040.0,3855.0,2.36,48.0,0.0,0.320037,0.31,1.450102,NaN
112,29,PTQ_Meas_160907.txt,Elasto(90),2.967190,387.9,-94.4,383.8,-94.4,239.10,107.30,...,17.999218,23040.0,3881.0,1.74,28.0,0.0,0.112564,0.05,1.144895,NaN
113,29,PTQ_Meas_160907.txt,Elasto(92),2.967190,263.4,-28.3,259.7,-22.0,140.85,39.30,...,17.999218,23040.0,3881.0,1.74,28.0,0.0,0.112564,0.05,1.144895,NaN
114,29,PTQ_Meas_160907.txt,Elasto(95),2.967190,237.6,-23.9,228.1,-23.9,126.00,46.20,...,17.999218,23040.0,3881.0,1.74,28.0,0.0,0.112564,0.05,1.144895,NaN


#### Analysiere Metrik Warnungen


In [17]:
# Filtere den DataFrame auf Zeilen, bei denen metrics_warning True ist
warnings_df = df[df['metrics_warning']]

# Gruppiere nach 'treatment' und 'sensor_name', zähle Warnungen und sammle IDs
warnings_summary = (
    warnings_df
    .groupby(['treatment', 'sensor_name'], observed=False)
    .agg(
        warning_count=('metrics_warning', 'size'),
         ids=('id', lambda x: sorted(x.astype(str).tolist()))
    )
    .reset_index()
)

warnings_summary = warnings_summary[warnings_summary['warning_count'] > 0]
warnings_summary

,treatment,sensor_name,warning_count,ids
9,cobra_static,Elasto(92),3,"[21, 23, 28]"
10,cobra_static,Elasto(95),5,"[22, 24, 25, 26, 27]"


In [18]:
# LaTeX-String erstellen
latex_string = warnings_summary.to_latex(
    index=False, 
    escape=True, 
    column_format="llll",  # Spaltenformat angepasst
    float_format="{:0.2f}".format
)

caption = "Plesse - Ergebnisse, Schwingung, Warnung Anpassungsgüte"
caption_long = "Plesse - Ergebnisse, Schwingung, Warnung Anpassungsgüte, r < 0,75"

save_latex_table(latex_string, caption, latex_export_directory, caption_long)

Content saved to: C:\kyellsen\005_Projekte\2024_BA\032_Feldversuch_2023_Plesse\030_Analysen\2023_Kronensicherung_Plesse_Kraefte_Schwingungen\working_directory\export_latex\plesse_ergebnisse_schwingung_warnung_anpassungsgute.tex


### EXPORT: Ungefilterte Daten exportieren für Externe (.feather, .csv)
 

In [19]:
df.to_feather(data_export_directory / "_dataset_full.feather")
df.to_csv(data_export_directory / "_dataset_full.csv", sep=";", index=True, encoding="utf-8")

## CLEANING: Daten bereinigen und Filtern

In [20]:
# Liste der Spalten, die beibehalten werden sollen (mit neuem Namen)
select_cols = [
    'id', 'rope_datetime', 'treatment', 'release_force_target',
    'rope_release', 'cable_max_force', 'cable_integral', 

    # umbenannt
    'sensor_name', 'location', 'direction', 'height', 'diameter',
    'max_strain', 'max_compression',
    'integral_intercept', 'integral_positiv', 'integral_negativ',
    'integral_abs', 'integral_ratio',
    'm_amplitude', 'm_amplitude_2',
    'initial_amplitude', 'damping_coeff', 'frequency_damped', 'phase_angle', 'y_shift', 'x_shift',
    'frequency_undamped', 'damping_ratio',
    'metrics_warning', 'pearson_r', 'nrmse', 'nmae'
]

df = (
    df
    # Auswahl der gewünschten Spalten
    [select_cols]
    # Filter auf gültige Kombinationen
    .query(
        "release_force_target in [2.0, 2.4, 2.8] "
        "and treatment in ['free', 'gefa_dynamic', 'cobra_static']"
    )
    .copy()
)

In [21]:
# Nicht verwendete Kategorien entfernen
df["treatment"] = df["treatment"].cat.remove_unused_categories()
df['cable_integral'] = df['cable_integral'].fillna(0)

In [22]:
df.head(10)

,id,rope_datetime,treatment,release_force_target,rope_release,cable_max_force,cable_integral,sensor_name,location,direction,...,frequency_damped,phase_angle,y_shift,x_shift,frequency_undamped,damping_ratio,metrics_warning,pearson_r,nrmse,nmae
4,2,2022-03-23 11:24:23,free,2.8,2.72310,NaN,0.0,Elasto(90),StB,elongation,...,0.441518,0.200000,-5.287164,0.116641,0.442936,0.503994,False,0.943158,0.038866,0.018011
5,2,2022-03-23 11:24:23,free,2.8,2.72310,NaN,0.0,Elasto(92),StB,elongation,...,0.432441,-0.200000,-14.233644,-0.009730,0.436601,0.873595,False,0.929554,0.034560,0.016183
6,2,2022-03-23 11:24:23,free,2.8,2.72310,NaN,0.0,Elasto(95),StA,elongation,...,0.422702,-0.200000,-19.011309,-0.123043,0.426319,0.823807,False,0.908058,0.044873,0.024274
7,2,2022-03-23 11:24:23,free,2.8,2.72310,NaN,0.0,Elasto(98),StA,elongation,...,0.443529,-0.065967,-1.099515,0.118608,0.445993,0.663224,False,0.980663,0.029621,0.021087
8,3,2022-03-23 11:31:12,free,2.8,2.76205,NaN,0.0,Elasto(90),StB,elongation,...,0.440653,0.200000,7.775284,0.097887,0.441317,0.344951,False,0.957380,0.043035,0.018619
9,3,2022-03-23 11:31:12,free,2.8,2.76205,NaN,0.0,Elasto(92),StB,elongation,...,0.439321,-0.075771,7.052210,0.094719,0.441380,0.608931,False,0.933463,0.043534,0.021255
10,3,2022-03-23 11:31:12,free,2.8,2.76205,NaN,0.0,Elasto(95),StA,elongation,...,0.436023,-0.200000,-3.785755,-0.075741,0.438127,0.618021,False,0.938307,0.039020,0.022498
11,3,2022-03-23 11:31:12,free,2.8,2.76205,NaN,0.0,Elasto(98),StA,elongation,...,0.441350,-0.200000,-17.747289,-0.203345,0.442888,0.525070,False,0.974139,0.028693,0.020715
12,4,2022-03-23 11:54:33,free,2.8,2.73950,NaN,0.0,Elasto(90),StB,elongation,...,0.439065,0.200000,-23.445975,0.120929,0.439764,0.354677,False,0.945320,0.049764,0.021509
13,4,2022-03-23 11:54:33,free,2.8,2.73950,NaN,0.0,Elasto(92),StB,elongation,...,0.436850,-0.200000,-19.454720,0.061912,0.438829,0.598701,False,0.922665,0.050822,0.020659


### EDIT: Datendokumentation an zusammengeführte und gefilterte Daten anpassen

Passe Datendokumentation den zusammengeführten und gefilterten Daten an, ergänze weitere später berechnete Features, erstellt vollstände aktuelle Datendokumentation

In [23]:
# Lade das Dictionary mit der Daten Dokumentation
with open(data_path / "calc_strain_data_dict.json", "r", encoding="utf-8") as f:
    calc_strain_data_dict = json.load(f)
    
select_cols += list(calc_strain_data_dict.keys())

In [24]:
# Rope-Keys erzeugen
ls3_rope_data_dict = {f"rope_{k}": v for k, v in ls3_data_dict.items()}
# Cable-Keys erzeugen
ls3_cable_data_dict = {    f"cable_{k}": v for k, v in ls3_data_dict.items()}

In [25]:
# Alles zusammenführen
full_data_dict = {
    **sensor_data_dict,
    **series_data_dict,
    **ls3_rope_data_dict,
    **ls3_cable_data_dict,
    **ptq_data_dict,
    **calc_strain_data_dict
}

In [26]:
# 1. sensor → Deutsch
if "sensor" in full_data_dict:
    full_data_dict["sensor"]["Deutsch"] = "Elastometer"

# 2. cable_max → Beschreibung, Deutsch, Zeichen
if "cable_max_force" in full_data_dict:
    full_data_dict["cable_max_force"]["Beschreibung"] = "Maximale gemessene Kraftspitze in der KS"
    full_data_dict["cable_max_force"]["Deutsch"] = "Kraftspitze KS"
    full_data_dict["cable_max_force"]["Zeichen"] = "$F_{\\mathrm{cable, max}}$"

In [27]:
data_dict = {key: full_data_dict[key] for key in select_cols if key in full_data_dict}
data_dict

{'id': {'Kategorie': 'ptq',
  'Zeichen': 'ID',
  'Deutsch': 'ID Messung',
  'Datentyp': 'int64',
  'Einheit': '-',
  'Beschreibung': 'Eindeutige ID der Messung'},
 'rope_datetime': {'Kategorie': 'ls3',
  'Zeichen': '$t$',
  'Deutsch': 'Zeitstempel',
  'Datentyp': 'object',
  'Einheit': '-',
  'Beschreibung': 'Startzeitpunkt der Messung laut Gerät'},
 'treatment': {'Kategorie': 'series',
  'Zeichen': 'treatment',
  'Deutsch': 'Behandlung',
  'Datentyp': 'category',
  'Einheit': '-',
  'Beschreibung': 'Art der KS: \\texttt{free}, \\texttt{gefa\\_dynamic}, \\texttt{cobra\\_static}'},
 'release_force_target': {'Kategorie': 'series',
  'Zeichen': '$F_{\\mathrm{release,target}}$',
  'Deutsch': 'Vorspannkraft-Soll',
  'Datentyp': 'float64',
  'Einheit': 'kN',
  'Beschreibung': 'Geplante Vorspannkraft im Zugseil bei Release'},
 'rope_release': {'Kategorie': 'ls3',
  'Zeichen': '$F_{\\mathrm{release}}$',
  'Deutsch': 'Vorspannkraft-Ist',
  'Datentyp': 'float64',
  'Einheit': 'kN',
  'Beschreibu

In [28]:
df.head(100)

,id,rope_datetime,treatment,release_force_target,rope_release,cable_max_force,cable_integral,sensor_name,location,direction,...,frequency_damped,phase_angle,y_shift,x_shift,frequency_undamped,damping_ratio,metrics_warning,pearson_r,nrmse,nmae
4,2,2022-03-23 11:24:23,free,2.8,2.72310,NaN,0.000000,Elasto(90),StB,elongation,...,0.441518,0.200000,-5.287164,0.116641,0.442936,0.503994,False,0.943158,0.038866,0.018011
5,2,2022-03-23 11:24:23,free,2.8,2.72310,NaN,0.000000,Elasto(92),StB,elongation,...,0.432441,-0.200000,-14.233644,-0.009730,0.436601,0.873595,False,0.929554,0.034560,0.016183
6,2,2022-03-23 11:24:23,free,2.8,2.72310,NaN,0.000000,Elasto(95),StA,elongation,...,0.422702,-0.200000,-19.011309,-0.123043,0.426319,0.823807,False,0.908058,0.044873,0.024274
7,2,2022-03-23 11:24:23,free,2.8,2.72310,NaN,0.000000,Elasto(98),StA,elongation,...,0.443529,-0.065967,-1.099515,0.118608,0.445993,0.663224,False,0.980663,0.029621,0.021087
8,3,2022-03-23 11:31:12,free,2.8,2.76205,NaN,0.000000,Elasto(90),StB,elongation,...,0.440653,0.200000,7.775284,0.097887,0.441317,0.344951,False,0.957380,0.043035,0.018619
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99,25,2022-03-23 16:44:31,cobra_static,2.4,2.45465,2.80,1.455661,Elasto(98),StA,elongation,...,0.526233,0.200000,0.004065,-0.123942,0.530764,0.826263,False,0.939117,0.040525,0.028335
100,26,2022-03-23 16:56:04,cobra_static,2.0,1.99840,2.33,1.334834,Elasto(90),StB,elongation,...,0.543590,-0.200000,-5.140277,0.088797,0.546791,0.682871,False,0.890898,0.044373,0.022288
101,26,2022-03-23 16:56:04,cobra_static,2.0,1.99840,2.33,1.334834,Elasto(92),StB,elongation,...,0.350000,-0.200000,-2.964533,-0.079053,0.384487,2.857143,False,0.866183,0.057465,0.033882
102,26,2022-03-23 16:56:04,cobra_static,2.0,1.99840,2.33,1.334834,Elasto(95),StA,elongation,...,0.466347,-0.200000,4.559008,-0.153434,0.492757,2.144327,True,0.747046,0.070126,0.040196


In [29]:
# In DataFrame umwandeln
data_dict_df =  build_data_dict_df(data_dict)

# In Markdown umwandeln und anzeigen
md_text = data_dict_df.to_markdown(tablefmt="github")
display(Markdown(md_text))

|    | Variable                | Kategorie       | Zeichen                        | Deutsch                              | Datentyp   | Einheit   | Beschreibung                                                              |
|----|-------------------------|-----------------|--------------------------------|--------------------------------------|------------|-----------|---------------------------------------------------------------------------|
|  0 | id                      | ptq             | ID                             | ID Messung                           | int64      | -         | Eindeutige ID der Messung                                                 |
|  1 | rope_datetime           | ls3             | $t$                            | Zeitstempel                          | object     | -         | Startzeitpunkt der Messung laut Gerät                                     |
|  2 | treatment               | series          | treatment                      | Behandlung                           | category   | -         | Art der KS: \texttt{free}, \texttt{gefa\_dynamic}, \texttt{cobra\_static} |
|  3 | release_force_target    | series          | $F_{\mathrm{release,target}}$  | Vorspannkraft-Soll                   | float64    | kN        | Geplante Vorspannkraft im Zugseil bei Release                             |
|  4 | rope_release            | ls3             | $F_{\mathrm{release}}$         | Vorspannkraft-Ist                    | float64    | kN        | Tatsächlich realisierte Vorspannkraft im Zugseil bei Release              |
|  5 | cable_max_force         | ls3             | $F_{\mathrm{cable, max}}$      | Kraftspitze KS                       | float      | kN        | Maximale gemessene Kraftspitze in der KS                                  |
|  6 | cable_integral          | ls3             | $\int F\,dt$                   | Kraftintegral                        | float      | kN·s      | Zeitliches Integral der Kraft (Fläche unter der Kraft-Zeit-Kurve)         |
|  7 | sensor_name             | ptq             | sensor                         | Sensorname                           | object     | -         | Bezeichnung des Elastometers                                              |
|  8 | location                | sensor_position | location                       | Position                             | string     | -         | Position des Sensors am Stamm                                             |
|  9 | direction               | sensor_position | direction                      | Richtung                             | string     | -         | Zug- oder Druckseite                                                      |
| 10 | height                  | sensor_position | $h$                            | Höhe                                 | Float64    | m         | Höhe des Sensors am Stamm                                                 |
| 11 | diameter                | sensor_position | $d$                            | Durchmesser                          | Float64    | cm        | Durchmesser des Stammes                                                   |
| 12 | max_strain              | ptq             | $\Delta L_{\mathrm{max}}$      | Dehnung max. gemessen                | float64    | $\mu$m    | Gemessene maximale absolute Randfaserdehnung                              |
| 13 | max_compression         | ptq             | $\Delta L_{\mathrm{comp,max}}$ | Stauchung max. gemessen              | float64    | $\mu$m    | Gemessene maximale absolute Randfaserstauchung                            |
| 14 | m_amplitude             | ptq_osc         | $mA$                           | Manuelle Amplitude                   | float64    | $\mu$m    | Manuell berechnete Amplitude über den Schwingungsabschnitt                |
| 15 | m_amplitude_2           | ptq_osc         | $mA_2$                         | Manuelle Amplitude 2                 | float64    | $\mu$m    | Manuell berechnete Amplitude zwischen 2. Peak und Minimum                 |
| 16 | initial_amplitude       | ptq_osc         | $A$                            | Anfangsamplitude                     | float64    | $\mu$m    | Initiale Amplitude der angepassten Schwingung                             |
| 17 | damping_coeff           | ptq_osc         | $\delta$                       | Dämpfungskoeffizient                 | float64    | 1/s       | Koeffizient der exponentiellen Dämpfung                                   |
| 18 | frequency_damped        | ptq_osc         | $f_{\mathrm{d}}$               | Gedämpfte Frequenz                   | float64    | Hz        | Frequenz der gedämpften Schwingung                                        |
| 19 | phase_angle             | ptq_osc         | $\varphi$                      | Phasenwinkel                         | float64    | rad       | Anfangsphase der Schwingung                                               |
| 20 | y_shift                 | ptq_osc         | $y_0$                          | Vertikaler Versatz                   | float64    | $\mu$m    | Vertikaler Offset der Schwingung                                          |
| 21 | x_shift                 | ptq_osc         | $t_0$                          | Zeitverschiebung                     | float64    | s         | Horizontale Verschiebung der Schwingung                                   |
| 22 | frequency_undamped      | ptq_osc         | $f_0$                          | Ungedämpfte Frequenz                 | float64    | Hz        | Frequenz der ungedämpften Schwingung                                      |
| 23 | damping_ratio           | ptq_osc         | $D$                            | Dämpfungsgrad                        | float64    | -         | Verhältnis von Dämpfung zu Frequenz                                       |
| 24 | metrics_warning         | ptq_osc         | warning                        | Fit-Warnung                          | bool       | -         | Warnung, wenn Qualitätsmetrik Schwellenwerte unterschreitet               |
| 25 | pearson_r               | ptq_osc_metric  | $r$                            | Pearson-Korrelation                  | float64    | -         | Korrelationskoeffizient der Anpassung                                     |
| 26 | nrmse                   | ptq_osc_metric  | $\mathrm{NRMSE}$               | Normalisierter RMSE                  | float64    | -         | Normalisierter mittlerer quadratischer Fehler                             |
| 27 | nmae                    | ptq_osc_metric  | $\mathrm{NMAE}$                | Normalisierter MAE                   | float64    | -         | Normalisierter mittlerer absoluter Fehler                                 |
| 28 | calc_max_strain         | calc_strain     | $\Delta L_{\mathrm{calc,max}}$ | Dehnung max. berechnet               | float64    | $\mu$m    | Berechnete maximale Randfaserdehnung                                      |
| 29 | calc_max_strain_relativ | calc_strain     | $\Delta L_{\mathrm{calc,rel}}$ | Berechnete maximale Randfaserdehnung | float64    | $\mu$m    | Differenz gemessene und berechnete maximale Randfaserdehnung              |
| 30 | strain_difference       | calc_strain     | $\Delta \varepsilon$           | Differenz der Faserdehnung           | float64    | \%        | Relative Differenz der berechneten und gemessenen max. Dehnung            |

### EXPORT: Gefilterte Daten und Datendokumentation exportieren für Weiterverarbeitung (.feather, .csv, .json)

In [30]:
df.to_feather(data_export_directory / "_dataset_clean.feather")
df.to_csv(data_export_directory / "_dataset_clean.csv", sep=";", index=True, encoding="utf-8")

with open(data_export_directory / "_data_dict_clean.json", "w", encoding="utf-8") as f:
    json.dump(data_dict, f, indent=4, ensure_ascii=False)

### LATEX-EXPORT: Datendokumentation als Latex-Tabelle exportieren (.tex)

In [31]:
# Erzeuge DataFrame für LaTeX mit einheitlichem Aufbau
data_dict_df = build_data_dict_df(data_dict, escape_index=True, select_latex_fields=True)

# Exportiere als LaTeX
latex_string = data_dict_df.to_latex(index=False, escape=False)

caption = "Plesse - Ergebnisse, Daten Dokumentation"
caption_long = "Plesse - Ergebnisse, Daten Dokumentation, Kräfte, Dehnungen und Schwingungsparameter"

save_latex_table(latex_string, caption, latex_export_directory, caption_long=caption_long)

Content saved to: C:\kyellsen\005_Projekte\2024_BA\032_Feldversuch_2023_Plesse\030_Analysen\2023_Kronensicherung_Plesse_Kraefte_Schwingungen\working_directory\export_latex\plesse_ergebnisse_daten_dokumentation.tex
